# Tests MSSQL Python

Frist create a .env file:

```env
MSSQL_HOST="..."
MSSQL_PORT=1433
MSSQL_DATABASE="..."
MSSQL_SCHEMA="dbo"
MSSQL_USERNAME="..."
MSSQL_PASSWORD="..."
```

In [8]:
%reload_ext dotenv
%dotenv

import os

config = {
    "host": os.getenv("MSSQL_HOST"),
    "port": int(os.getenv("MSSQL_PORT") or 0),
    "database": os.getenv("MSSQL_DATABASE"),
    "schema": os.getenv("MSSQL_SCHEMA") or "dbo",
    "username": os.getenv("MSSQL_USERNAME"),
    "password": os.getenv("MSSQL_PASSWORD"),
}

## PyODBC

**Microsoft ODBC Driver for SQL Server is required before to install pyodbc**.

Please follow the instructions to install the driver for your OS. See the [docs](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server) for details.

To install run this command:
```bash
poetry add pyodbc
```

In [9]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["host"]};PORT={config["port"]};'
    f'DATABASE={config["database"]};'
    f'UID={config["username"]};'
    f'PWD={config["password"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        rows = cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA='dbo' AND TABLE_TYPE='BASE TABLE';")
        display(rows.fetchall())

'DRIVER={SQL Server};SERVER=tcp:ap50-tst-datal-sqlmi01.33b9b5988eba.database.windows.net;PORT=1433;DATABASE=Dictionary_VN;UID=datafactory;PWD=7TE7rxMnTwM9V'

[('Dictionary_VN', 'dbo', 'Dic_Customer', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'ZetaDate-2019-2020', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'ZetaDate-2021-2023', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'U_Dic_Territory_Bonus', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'ZetaData_Market', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_Activity2Score', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'U_Dic_Territory', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_ActivityPayRoll', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_EventCost', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Raw_DKSH', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Dic_Product', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Corner_Reference', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_Specialty_1', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Dic_Parameter', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_HCP_Type', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Ref_HCO_Type', 'BASE TABLE'),
 ('Dictionary_VN', 'dbo', 'Dic_ProductPrice'

## PyTDS

To install run this command:

```bash
poetry add python-tds
```

If you want to use TLS you should also install pyOpenSSL package:

```bash
poetry add pyOpenSSL
```

For a better performance install bitarray package too:

```bash
poetry add bitarray
```

To use Kerberos on non-Windows platforms (experimental) install kerberos package:

```bash
poetry add kerberos
```

## MiniAB

**miniab** requires duckdb and pyarrow and they should be installed before to use it.

To install duckdb and pyarrow run this command:

```bash
poetry add duckdb pyarrow
```

In [10]:
import duckdb
import miniab as mab

display(mab.get_available_connectors())

source: mab.Source = mab.get_source(
    "source-mssql",
    config = config,
    sync = False
)

source.select_all_streams()
result = source.read()

tteam_tbl = result.to_arrow("TTeam")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 100;").show()

['source-snowflake', 'source-mssql']

┌────────┬────────────────┬────────────┬───────────┐
│ TeamID │    TeamName    │ JobTitleID │ VersionID │
│ int32  │    varchar     │   int32    │   int32   │
├────────┼────────────────┼────────────┼───────────┤
│      2 │ EMERALD        │          5 │         1 │
│      3 │ RUBY-PROVINCES │          5 │         1 │
│      4 │ SAPHIRE        │          5 │         1 │
│      5 │ TOPAZ          │          5 │         1 │
│      6 │ JADE           │          5 │         1 │
│      7 │ JASPER         │          5 │         1 │
│      8 │ RUBY           │          5 │         1 │
│     10 │ PKAS           │          8 │         1 │
│     22 │ KAS            │          4 │         1 │
│     24 │ HKAS           │          7 │         1 │
│      · │  ·             │          · │         · │
│      · │  ·             │          · │         · │
│      · │  ·             │          · │         · │
│    146 │ KAS            │          4 │        23 │
│    147 │ HKAS           │          7 │      

## PyAirByte

PyAirByte requires docker and it should be installed before to use it.

To install PyAirByte run this command:

```bash
poetry add airbyte
```

In [ ]:
import duckdb
import airbyte as ab

display(ab.get_available_connectors())

source: ab.Source = ab.get_source(
    "source-mssql",
    config = config
)

source.select_all_streams()
result = source.read()

tteam_tbl = result.to_arrow("")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 10;").show()